In [1]:
import dask
import dask.dataframe as ddf
import os
from glob import glob
import re
import pandas as pd
import numpy as np
import pickle

/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
df = ddf.read_parquet('../../../../intermediate-result-new-select-editors/editor-profile/amr-info-per-editor-per-page/')

In [3]:
df = df.compute()

In [4]:
with open('../../all-data-article-clustering/articles-100-threshold.pkl' ,'rb') as f:
    selected_articles = pickle.load(f)

In [5]:
selected_talks = {'Talk:' + i for i in selected_articles}

In [6]:
selected_articles.update(selected_talks)

In [7]:
df = df[df['page.title'].isin(selected_articles)]

In [8]:
def page_type(title):
    if title.startswith('Talk:'):
        return 'Talk'
    return 'Article'

In [9]:
df['page.type'] = df['page.title'].apply(page_type)

In [10]:
df = df.drop(columns={'dir0'})

In [11]:
article = df[df['page.type'] == 'Article']

In [12]:
article.columns = article.columns + '.Article'

In [13]:
article = article.rename(columns={'contributor.username.Article':'contributor.username'})

In [14]:
article = article.groupby('contributor.username').sum()

In [15]:
talk = df[df['page.type'] == 'Talk']

In [16]:
talk.columns = talk.columns + '.Talk'

In [17]:
talk = talk.rename(columns={'contributor.username.Talk':'contributor.username'})

In [18]:
talk = talk.groupby('contributor.username').sum()

In [19]:
article.index.name = ''

In [20]:
talk.index.name = ''

In [21]:
article = article.T

In [22]:
talk = talk.T

In [23]:
res = pd.concat([article,talk]).fillna(0)

In [25]:
res.to_parquet('result/amr-info-all-data')